In [20]:
from google.colab import files
uploaded = files.upload()

Saving archive (1).zip to archive (1) (1).zip


In [21]:
import os
print(os.listdir())

['.config', 'archive (1) (1).zip', 'loan_dataset', 'creditpathai.db', 'loan_preprocessed.csv', 'archive (1).zip', 'sample_data']


In [22]:
import zipfile

with zipfile.ZipFile("archive (1).zip", 'r') as zip_ref:
    zip_ref.extractall("loan_dataset")

print(os.listdir("loan_dataset"))

['Loan_Default.csv']


In [23]:
import pandas as pd

df = pd.read_csv("loan_dataset/Loan_Default.csv")

print("Dataset Shape:", df.shape)
df.head()

Dataset Shape: (148670, 34)


,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,...,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
1,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
2,24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,...,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,...,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,...,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0


In [32]:
df.shape
df.head()

,year,Gender,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,loan_amount,rate_of_interest,Interest_rate_spread,...,Secured_by,total_units,income,Credit_Score,age,LTV,Region,Security_Type,Status,dtir1
0,2019,Sex Not Available,type1,p1,l1,nopc,nob/c,116500,3.99,0.3904,...,home,1U,1740.0,758,25-34,98.728814,south,direct,1,45.0
1,2019,Male,type2,p1,l1,nopc,b/c,206500,3.99,0.3904,...,home,1U,4980.0,552,55-64,75.135870,North,direct,1,39.0
2,2019,Male,type1,p1,l1,nopc,nob/c,406500,4.56,0.2000,...,home,1U,9480.0,834,35-44,80.019685,south,direct,0,46.0
3,2019,Male,type1,p4,l1,nopc,nob/c,456500,4.25,0.6810,...,home,1U,11880.0,587,45-54,69.376900,North,direct,0,42.0
4,2019,Joint,type1,p1,l1,nopc,nob/c,696500,4.00,0.3042,...,home,1U,10440.0,602,25-34,91.886544,North,direct,0,39.0


In [33]:
# ===============================
# DATA LOADING + CLEANING
# ===============================

import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("loan_dataset/Loan_Default.csv")

print("Original Shape:", df.shape)

# Drop ID column if exists
if "ID" in df.columns:
    df = df.drop(columns=["ID"])

# Remove leakage columns if present
leak_cols = ["credit_type", "co-applicant_credit_type"]
df = df.drop(columns=[col for col in leak_cols if col in df.columns])

# Remove duplicates
df = df.drop_duplicates().reset_index(drop=True)
print("After Removing Duplicates:", df.shape)

# Handle missing values
num_cols = df.select_dtypes(include=['int64','float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

print("Missing Values After Cleaning:", df.isnull().sum().sum())

# Outlier removal using IQR (important numeric columns)
def remove_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return data[(data[column] >= lower) & (data[column] <= upper)]

important_cols = ["Credit_Score", "LTV", "dtir1"]

for col in important_cols:
    if col in df.columns:
        df = remove_outliers_iqr(df, col)

df = df.reset_index(drop=True)

print("After Outlier Removal:", df.shape)

# Save cleaned CSV
df.to_csv("loan_cleaned.csv", index=False)
print("Clean dataset saved as loan_cleaned.csv")

Original Shape: (148670, 34)
After Removing Duplicates: (148670, 31)
Missing Values After Cleaning: 0
After Outlier Removal: (137377, 31)
Clean dataset saved as loan_cleaned.csv


In [42]:
# ===============================
# SQLITE DATABASE STORAGE
# ===============================

import sqlite3

conn = sqlite3.connect("creditpathai.db")

# Store cleaned data
df.to_sql("clean_loans", conn, if_exists="replace", index=False)

# Check tables
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables inside database:")
print(tables)

conn.close()

print("Database integration completed successfully.")
leak_cols = [
    "Interest_rate_spread",
    "Upfront_charges",
    "rate_of_interest"
]

df = df.drop(columns=[col for col in leak_cols if col in df.columns])

Tables inside database:
              name
0        raw_loans
1  processed_loans
2      clean_loans
Database integration completed successfully.
